In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from scipy.stats import zscore

# HoloViz and Bokeh
import colorcet as cc
import holoviews as hv; hv.extension('bokeh')
from holoviews.plotting.links import RangeToolLink
from holoviews.operation.datashader import rasterize
from holoviews import Dataset
from bokeh.models import HoverTool, WheelZoomTool
import panel as pn; pn.extension(template='material')

# Neuro repo
from neurodatagen.eeg import generate_eeg_powerlaw
from hvneuro import download_file

In [ ]:
import mne

### Intake data

In [ ]:
# This dataset is 2.6 MB on disk
url = "https://physionet.org/files/eegmmidb/1.0.0/S001/S001R04.edf?download"
local_data_path = "../../data/"

# Will not download if already present at local_data_path
local_file_path = download_file(url, local_data_path)

In [ ]:
raw = mne.io.read_raw_edf(local_file_path, preload=True)
raw.info

In [ ]:
# preview the channel names, types, signal ranges, and uncompressed size
# raw.describe()

### Gather the real timeseries annotations and clean up

In [ ]:
# get initial time of experiment
orig_time = raw.annotations.orig_time

# get annotations into pd df
annotations_df = raw.annotations.to_data_frame()

# Ensure the 'onset' column is in UTC timezone
annotations_df['onset'] = annotations_df['onset'].dt.tz_localize('UTC')

annotations_df['start'] = (annotations_df['onset'] - orig_time).dt.total_seconds()
annotations_df['end'] = annotations_df['start'] + annotations_df['duration']


unique_descriptions = annotations_df['description'].unique()
color_map = dict(zip(unique_descriptions, cc.glasbey[:len(unique_descriptions)]))
annotations_df['color'] = annotations_df['description'].map(color_map)

# annotations_df.head()


### Clean channel names, set sensor positions, and reference data

In [ ]:
# clean up the channel names
raw.rename_channels(lambda s: s.strip("."));

In [ ]:
time = raw.times
channels = raw.ch_names

# get the EEG data (for this data set, all channels are EEG anyways)
eeg_indices = mne.pick_types(raw.info, eeg=True)
data = raw.get_data(picks=eeg_indices, units={"eeg":"uV"})

In [ ]:
max_ch_disp = 20  # max channels to initially display
max_t_disp = 20 # max time in seconds to initially display

annotation = hv.VSpan(19, 20).opts(color='yellow', alpha=.15) # example annotation (start, end) time

hover = HoverTool(tooltips=[
    ("Channel", "@channel"),
    ("Time", "$x s"),
    ("Amplitude", "$y µV")])

channel_curves = []
for channel, channel_data in zip(channels, data):
    ds = Dataset((time, channel_data, channel), ["Time", "Amplitude", "channel"])
    curve = hv.Curve(ds, "Time", ["Amplitude", "channel"], label=f'{channel}')
    curve.opts(
        color="black",
        line_width=1,
        subcoordinate_y=True,
        subcoordinate_scale=3,
        tools=[hover])
    channel_curves.append(curve)

eeg_viewer = (annotation * hv.Overlay(channel_curves, kdims="Channel"))
eeg_viewer = eeg_viewer.opts(
    xlabel="Time (s)",
    ylabel="Channel",
    show_legend=False,
    responsive=True,
    shared_axes=False,
    aspect=2,
    xlim=(time.min(), time.max()),
    backend_opts={
        "x_range.bounds": (time.min()-2, time.max()),
        "y_range.bounds": (-2, len(channels))})

y_positions = range(len(channels))
yticks = [(i, ich) for i, ich in enumerate(channels)]

z_data = zscore(data, axis=1)

minimap = rasterize(hv.Image((time, y_positions, z_data), ["Time (s)", "Channel"], "Amplitude (uV)"))

clim_mul = 3
minimap = minimap.opts(
    cmap="RdBu_r", colorbar=False, xlabel='', alpha=.5, yticks=[yticks[0], yticks[-1]],
    height=125, responsive=True, default_tools=[], shared_axes=False, clim=(-z_data.std()*clim_mul, z_data.std()*clim_mul))

# Create RangeToolLink between the minimap and the main EEG viewer 
# max_y_disp = np.max(data[max_ch_disp-1,:] + ((max_ch_disp-1) * offset))
RangeToolLink(minimap, eeg_viewer, axes=["x", "y"], boundsx=(0, 20), boundsy=(-1, 10))

eeg_app = pn.Column((eeg_viewer + minimap * annotation)
eeg_app = eeg_app.opts(merge_tools=False).cols(1), min_height=650).servable(
    target='main', title='EEG Viewer with HoloViz and Bokeh')
eeg_app

In [ ]:
y_positions

In [ ]:
yticks